In [54]:
import logging
from utils import capital2lower
import datetime
import re

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s')

TIMECODES_THRESHOLD_COUNT = 3
TIMECODE_PATTERN = r'(\d*:*\d+:+\d+)'
STRIP_CHARS = ' #$%&@()*+[\\]^_`{|}~--−–—'
DOTS_CHARS = '.,;:?!'

REPLASMENTS = [
    '---', 
    '--', 
    '===', 
    '==', 
    ' =', 
    '___', 
    '__', 
    '_ _ _', 
    '_ _', 
    ' _', 
    '\n-', 
    '\n=', 
    '\n_', 
    '\n -', 
    '\n =', 
    '\n _',
]

def clean_timecodes_text(text):
    for pattern in REPLASMENTS:
        text = text.replace(pattern, '')

    return text.strip()
    

def time_to_seconds(time_str):
    if time_str.count(':') == 1:
        format_str = '%M:%S'
        time_obj = datetime.datetime.strptime(time_str, format_str)
        total_seconds = time_obj.minute * 60 + time_obj.second
    elif time_str.count(':') == 2:
        format_str = '%H:%M:%S'
        time_obj = datetime.datetime.strptime(time_str, format_str)
        total_seconds = time_obj.hour * 3600 + time_obj.minute * 60 + time_obj.second
    else:
        raise ValueError("Time format not recognized")
    return total_seconds
    

def get_timecodes_block_text(text):
    if not text:
        return ''
        
    for part in text.split('\n\n'):
        matches = re.findall(TIMECODE_PATTERN, part)
        if len(matches) > TIMECODES_THRESHOLD_COUNT:
            return part


def get_timecodes(text):
    timecodes_block_text = get_timecodes_block_text(text)
    timecodes_block_text = clean_timecodes_text(timecodes_block_text)

    timecodes = []
    for row in timecodes_block_text.split('\n'):
        if not (matched := re.findall(TIMECODE_PATTERN, row)):
            continue

        timecode_raw = matched[0]
        title = row.replace(timecode_raw, '')
        title = title.strip(STRIP_CHARS).lstrip(DOTS_CHARS)
        
        timecodes.append({
            'timecode': time_to_seconds(timecode_raw),
            'timecode_row': timecode_raw,
            'title': title,
        })
    
    return timecodes


In [55]:
import pathlib

samples = []

for file_path in pathlib.Path('samples').iterdir():
    samples.append(file_path.read_text())

for sample in samples:
    timecodes = get_timecodes(sample)
    print('🥗')
    print('timecodes: ')
    print(timecodes)
    print()
    

🥗
timecodes: 
[{'timecode': 0, 'timecode_row': '00:00', 'title': 'Игорь Мандель. Вступительное слово'}, {'timecode': 139, 'timecode_row': '02:19', 'title': 'Андрей Илларионов. Чем занимался и чем занимаюсь'}, {'timecode': 244, 'timecode_row': '04:04', 'title': 'Свободная экономика начинается со свободного общества'}, {'timecode': 332, 'timecode_row': '05:32', 'title': 'Большинство решений люди принимают исходя из их мировоззрения'}, {'timecode': 404, 'timecode_row': '06:44', 'title': 'Роль культуры'}, {'timecode': 523, 'timecode_row': '08:43', 'title': 'Институт Катона, Индекс человеческой свободы'}, {'timecode': 635, 'timecode_row': '10:35', 'title': 'Свободное общество только тогда чего-то стОит, если оно способно защищаться'}, {'timecode': 759, 'timecode_row': '12:39', 'title': 'Свобода в США под угрозой'}, {'timecode': 815, 'timecode_row': '13:35', 'title': 'Предмет личного интереса и исследований – свободное общество'}, {'timecode': 888, 'timecode_row': '14:48', 'title': 'Опасност